In [1]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [2]:
import yaml, matplotlib, random, pickle, time
import numpy as np
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import morlet2, cwt
from sklearn.mixture import GaussianMixture

from helper import _rotational, angle_calc

# Start Analysis

In [3]:
with open("config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [112]:
# Initialize
files_ref, bad_frames_ref = {}, {}
frame_start = 0
bp_list, scale_list, angles_list, power_list = [], [], [], []

# Bodypoints Used for Analysis
bp_analyze = []
for angle_points in config["angles"]:
    bp_analyze.extend(angle_points.values())
bp_analyze.append(config["bp_rotate"])
bp_analyze.extend(config["bp_scale"])
bp_analyze.append(config["bp_center"])
bp_analyze = np.unique(bp_analyze)

for path in tqdm(glob(f"{config['input_data_path']}/**/*.h5")):
    # Import
    store = pd.HDFStore(path, mode='a')
    df = store['df_with_missing']
    x_data = df.xs('x', level="coords", axis=1).to_numpy()
    y_data = df.xs('y', level="coords", axis=1).to_numpy()
    likelihood = df.xs('likelihood', level="coords", axis=1).to_numpy()
    store.close()
    
    # Check Likelihood
    below_thresh = np.where(likelihood[:, bp_analyze] < .2)
    fr_bad = np.unique(below_thresh[0])
    bad_frames_ref[path] = fr_bad

    # Center
    x_center = x_data[:,config['bp_center']]
    y_center = y_data[:,config['bp_center']]
    x_data -= x_center[:,np.newaxis]
    y_data -= y_center[:,np.newaxis]
        
    # Format
    DLC_data = np.concatenate((
        np.expand_dims(x_data, axis=-1), 
        np.expand_dims(y_data, axis=-1)), axis=-1)
    num_fr,_,_ = DLC_data.shape
    
    # Scale
    x_d = DLC_data[:,config['bp_scale'][0],0] - DLC_data[:,config['bp_scale'][1],0]
    y_d = DLC_data[:,config['bp_scale'][0],1] - DLC_data[:,config['bp_scale'][1],1]
    dist = np.sqrt(x_d**2+y_d**2)
    norm = np.median(dist)
    scale_list.append(norm)
    DLC_data /= norm

    # Rotate
    ROT_data, body_angle = _rotational(data=DLC_data, axis_bp=config['bp_rotate'])
    bp_list.append(ROT_data)

    # Angles
    angles = angle_calc(ROT_data, config['angles'])
    angles -= np.mean(angles, axis=0)
    angles_list.append(angles)

    # Record Files
    files_ref[path] = (frame_start, frame_start+num_fr)
    frame_start += num_fr

# Combine Bodypoints and Angles Data
tot_bp = np.concatenate(bp_list, axis=0)
tot_angles = np.concatenate(angles_list, axis=0)  

for angles in angles_list:
    # Normalize Angles
    angles -= np.mean(tot_angles, axis=0)
    
    # Morlet Wavelet
    num_fr, num_ang = angles.shape
    power = np.zeros((num_ang, config['f_bin'], num_fr))
    max_freq, min_freq = config['fps']/2, 1 # Nyquist Frequency
    freq = max_freq*2**(-1*np.log2(max_freq/min_freq)*
        (np.arange(config['f_bin'],0,-1)-1)/(config['f_bin']-1))
    widths = config['w']*config['fps'] / (2*freq*np.pi)
    
    # Normalization Factor
    s = (config['w'] + np.sqrt(2+config['w']**2))/(4*np.pi*freq)
    C = np.pi**(-0.25)*np.exp(0.25*(config['w']-np.sqrt(config['w']**2+2))**2)/np.sqrt(2*s)

    for i in range(num_ang):
        cwtm = cwt(angles[:,i], morlet2, widths, dtype=None, w=config['w'])
        # power[i] = np.abs(cwtm)**2
        power[i] = (np.abs(cwtm/np.expand_dims(np.sqrt(s),1)))/np.expand_dims(C, axis=(0,2))
    power_list.append(power)

tot_pwr = np.concatenate(power_list, axis=2)

# Take Out Bad Frames
tot_fr_bad = []
num_ang, num_freq, num_fr = tot_pwr.shape
for path, fr_range in files_ref.items():
    tot_fr_bad.extend(bad_frames_ref[path]+fr_range[0])
tot_fr_good = np.delete(np.arange(num_fr), tot_fr_bad)
good_tot_pwr = np.delete(tot_pwr, tot_fr_bad, axis=2)

# Dimensional Reduction
num_angles, num_freq, num_good_fr = good_tot_pwr.shape
power_mod = good_tot_pwr.reshape((num_angles*num_freq, num_good_fr)).T

In [138]:
import os

In [145]:
np_embed.shape

NameError: name 'np_embed' is not defined

In [137]:
tot_bp

array([[[-5.63538396e-02,  2.15802390e+00],
        [-5.55111512e-17,  1.00591116e+00],
        [ 0.00000000e+00,  0.00000000e+00],
        ...,
        [-1.95039306e+00, -3.21638279e+00],
        [-6.99798077e-01, -5.98947134e-01],
        [-1.65348812e-01, -5.98785161e-01]],

       [[-1.34454807e-01,  2.14554048e+00],
        [ 2.77555756e-17,  9.90965760e-01],
        [ 0.00000000e+00,  0.00000000e+00],
        ...,
        [-1.76421906e+00, -4.00727230e+00],
        [-8.62660491e-01, -1.31930830e+00],
        [-2.81797968e-01, -7.40630232e-01]],

       [[-3.64752869e-02,  2.13309850e+00],
        [ 5.20417043e-17,  9.74703865e-01],
        [ 0.00000000e+00,  0.00000000e+00],
        ...,
        [-2.25651798e+00, -4.36505033e+00],
        [-1.21874824e+00, -1.73118647e+00],
        [-3.43831963e-01, -6.27632117e-01]],

       ...,

       [[ 4.68174817e-01,  2.03519666e+00],
        [ 5.55111512e-17,  1.08140041e+00],
        [ 0.00000000e+00,  0.00000000e+00],
        ...,
     